# Philly Evictions Analysis
## Environment Setup

In [1]:
import pandas as pd
import numpy as np
import pipeline

pipeline.notebook.set_up()

## Load Data
Our evictions data has already been augmented with data from the ACS and from
Philadelphia's open data portal. We load in the final merged dataset.

In [2]:
df = pd.read_csv('data/final_merged_df.csv')
df.head()

,GEOID,year_evictions,evictions,low-flag,imputed,subbed,evictions_t-1,evictions_t-2,evictions_t-5,crime_count,...,units,occupied_units,vacant_units,for_rent_units,num_af_am_alone,num_hisp,black_alone_owner_occupied,num_with_high_school_degree,num_with_ged,num_unemployed
0,421010001001,2010,13.0,0,0,1,8.0,10.0,9.0,NaN,...,220.0,189.0,94.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,421010001001,2011,7.0,0,0,1,13.0,8.0,3.0,NaN,...,1173.0,934.0,728.0,38.0,76.0,94.0,14.0,NaN,NaN,NaN
2,421010001001,2012,7.0,0,0,1,7.0,13.0,13.0,351.0,...,1347.0,1024.0,790.0,124.0,104.0,111.0,12.0,NaN,NaN,0.0
3,421010001001,2013,9.0,0,0,1,7.0,7.0,10.0,607.0,...,1481.0,1094.0,826.0,139.0,88.0,122.0,12.0,52.0,0.0,0.0
4,421010001001,2014,4.0,0,0,1,9.0,7.0,8.0,395.0,...,1489.0,1126.0,823.0,155.0,89.0,57.0,12.0,53.0,0.0,0.0


## Split Data By Year
We have data for 2009 to 2016. We want to split this data into training set /
test set pairs using a temporal cross-validation approach.

In [3]:
splits = pipeline.split_by_year(df, colname='year_evictions')
pipeline.split_boundaries(splits, colname='year_evictions')

,train_start,train_end,test_start,test_end
0,2010,2010,2011,2011
1,2011,2011,2012,2012
2,2012,2012,2013,2013
3,2013,2013,2014,2014
4,2014,2014,2015,2015
5,2015,2015,2016,2016


## Data Cleaning
We want to clean each of our training set / test set pairs. We use a function
called `clean_split()` that cleans both sets at once, making sure to clean the
test data using the same bins and categories applied to the training data.

In [4]:
%psource pipeline.clean_split

def clean_split(split):
    train_df, test_df = split
    train_df = clean_overall_data(train_df)
    test_df = clean_overall_data(test_df)

    features_generator = get_feature_generators(train_df)
    train_df, test_df = \
        clean_and_create_features(train_df, test_df, features_generator)

    return train_df, test_df



In [5]:
cleaned_splits = [pipeline.clean_split(split) for split in splits]

/home/sinclairtarget/harris/ml/philly-evictions/.venv/lib/python3.7/site-packages/sklearn/utils/extmath.py:747: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/sinclairtarget/harris/ml/philly-evictions/.venv/lib/python3.7/site-packages/sklearn/utils/extmath.py:688: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)
/home/sinclairtarget/harris/ml/philly-evictions/.venv/lib/python3.7/site-packages/sklearn/utils/extmath.py:747: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/sinclairtarget/harris/ml/philly-evictions/.venv/lib/python3.7/site-packages/sklearn/utils/extmath.py:688: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)
/home/sinclairtarget/harris/ml/philly-evictions/.venv/lib/python3.7/site-packages/sklearn/utils/extmath.py:747: RuntimeWarning: invalid value enco

## Data Labeling
We plan to use both regression-based models and binary classifiers. For our
binary classifiers, we will need to label our data using a binary label.

Our binary label separates block groups into two classes: "high" and "low"
eviction rate block groups. The "high" eviction rate block groups are those
that we believe should be prioritized for intervention.

Any block group with more than 14 evictions is considered a "high" eviction
rate block group. Roughly 16% of Philadelphia block groups are "high" eviction
rate block groups. We have picked this lower boundary because we know that
Philadelphia can afford to target about 16% of block groups for intervention.

In [6]:
labeled_splits = [pipeline.label(split, lower_bound=14, drop_column=True)
                  for split in cleaned_splits]

## Model Generation
### Binary Classifiers
Our binary classifiers are given by the following list:

In [7]:
pipeline.clfs

{'LR': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=100,
                    multi_class='warn', n_jobs=None, penalty='l2',
                    random_state=1234, solver='liblinear', tol=0.0001, verbose=0,
                    warm_start=False),
 'KNN': KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                      weights='uniform'),
 'DT': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                        max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort=False,
                        random_state=1234, splitter='best'),
 'SVM': SVC(C=1.0, cache_size=200, class_w

We plan to run a grid search using the following hyperparameters.

In [8]:
pipeline.clf_small_grid

{'LR': {'penalty': ['l1', 'l2'], 'C': [0.01, 0.1]},
 'KNN': {'n_neighbors': [5, 10],
  'weights': ['uniform', 'distance'],
  'algorithm': ['auto', 'ball_tree', 'kd_tree']},
 'DT': {'criterion': ['gini', 'entropy'],
  'max_depth': [5, 50],
  'max_features': [None],
  'min_samples_split': [5, 10]},
 'SVM': {'C': [0.01, 0.1]},
 'RF': {'n_estimators': [100, 1000],
  'max_depth': [5, 50],
  'max_features': ['sqrt', 'log2'],
  'min_samples_split': [5, 10]},
 'GB': {'n_estimators': [100, 1000],
  'learning_rate': [0.01, 0.05],
  'subsample': [0.1, 0.5],
  'max_depth': [5, 10]},
 'AB': {'algorithm': ['SAMME', 'SAMME.R'], 'n_estimators': [100, 1000]},
 'NB': {},
 'ET': {'n_estimators': [100, 1000],
  'criterion': ['gini', 'entropy'],
  'max_depth': [5, 10],
  'max_features': ['sqrt', 'log2'],
  'min_samples_split': [5, 10]},
 'BC': {'n_estimators': [100, 1000]}}

We also want to evaluate our models at the following thresholds:

In [9]:
thresholds = [12, 16, 20]

We run our models for each of our splits.

In [10]:
models = pipeline.clfs
grid = pipeline.clf_small_grid

results_df = pd.DataFrame(columns=[
    'split',
    'classifier',
    'parameters',
    'threshold'
] + pipeline.evaluate.ClassifierEvaluator.metric_names())

for i, (train_df, test_df) in enumerate(labeled_splits, start=1):
    train_df = train_df.drop(columns=['GEOID', 'year_evictions'])
    test_df = test_df.drop(columns=['GEOID', 'year_evictions'])
    df = pipeline.run_clf_loop(
        test_df, train_df, models, grid, 'label', thresholds, debug=True
    )

    df = df.assign(split=i)
    results_df = results_df.append(df, ignore_index=True)

results_df.to_csv('results/clf-small-grid.csv', index=False)

Running LR with parameters {'C': 0.01, 'penalty': 'l1'}
Running LR with parameters {'C': 0.01, 'penalty': 'l2'}
Running LR with parameters {'C': 0.1, 'penalty': 'l1'}
Running LR with parameters {'C': 0.1, 'penalty': 'l2'}
Running KNN with parameters {'algorithm': 'auto', 'n_neighbors': 5, 'weights': 'uniform'}
Running KNN with parameters {'algorithm': 'auto', 'n_neighbors': 5, 'weights': 'distance'}
Running KNN with parameters {'algorithm': 'auto', 'n_neighbors': 10, 'weights': 'uniform'}
Running KNN with parameters {'algorithm': 'auto', 'n_neighbors': 10, 'weights': 'distance'}
Running KNN with parameters {'algorithm': 'ball_tree', 'n_neighbors': 5, 'weights': 'uniform'}
Running KNN with parameters {'algorithm': 'ball_tree', 'n_neighbors': 5, 'weights': 'distance'}
Running KNN with parameters {'algorithm': 'ball_tree', 'n_neighbors': 10, 'weights': 'uniform'}
Running KNN with parameters {'algorithm': 'ball_tree', 'n_neighbors': 10, 'weights': 'distance'}
Running KNN with parameters {'

Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 100}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 1000}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 100}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 1000}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 5, 'n_estimators': 100}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 5, 'n_estimators': 1000}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 10, 'n_estimators': 100}
Running ET with parameters {'criter

/home/sinclairtarget/harris/ml/philly-evictions/.venv/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Running LR with parameters {'C': 0.01, 'penalty': 'l1'}
Running LR with parameters {'C': 0.01, 'penalty': 'l2'}
Running LR with parameters {'C': 0.1, 'penalty': 'l1'}
Running LR with parameters {'C': 0.1, 'penalty': 'l2'}
Running KNN with parameters {'algorithm': 'auto', 'n_neighbors': 5, 'weights': 'uniform'}
Running KNN with parameters {'algorithm': 'auto', 'n_neighbors': 5, 'weights': 'distance'}
Running KNN with parameters {'algorithm': 'auto', 'n_neighbors': 10, 'weights': 'uniform'}
Running KNN with parameters {'algorithm': 'auto', 'n_neighbors': 10, 'weights': 'distance'}
Running KNN with parameters {'algorithm': 'ball_tree', 'n_neighbors': 5, 'weights': 'uniform'}
Running KNN with parameters {'algorithm': 'ball_tree', 'n_neighbors': 5, 'weights': 'distance'}
Running KNN with parameters {'algorithm': 'ball_tree', 'n_neighbors': 10, 'weights': 'uniform'}
Running KNN with parameters {'algorithm': 'ball_tree', 'n_neighbors': 10, 'weights': 'distance'}
Running KNN with parameters {'

Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 100}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 1000}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 100}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 1000}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 5, 'n_estimators': 100}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 5, 'n_estimators': 1000}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 10, 'n_estimators': 100}
Running ET with parameters {'criter

...

Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 100}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 1000}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 100}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 1000}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 5, 'n_estimators': 100}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 5, 'n_estimators': 1000}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 10, 'n_estimators': 100}
Running ET with parameters {'criter

/home/sinclairtarget/harris/ml/philly-evictions/.venv/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Running LR with parameters {'C': 0.01, 'penalty': 'l1'}
Running LR with parameters {'C': 0.01, 'penalty': 'l2'}
Running LR with parameters {'C': 0.1, 'penalty': 'l1'}
Running LR with parameters {'C': 0.1, 'penalty': 'l2'}
Running KNN with parameters {'algorithm': 'auto', 'n_neighbors': 5, 'weights': 'uniform'}
Running KNN with parameters {'algorithm': 'auto', 'n_neighbors': 5, 'weights': 'distance'}
Running KNN with parameters {'algorithm': 'auto', 'n_neighbors': 10, 'weights': 'uniform'}
Running KNN with parameters {'algorithm': 'auto', 'n_neighbors': 10, 'weights': 'distance'}
Running KNN with parameters {'algorithm': 'ball_tree', 'n_neighbors': 5, 'weights': 'uniform'}
Running KNN with parameters {'algorithm': 'ball_tree', 'n_neighbors': 5, 'weights': 'distance'}
Running KNN with parameters {'algorithm': 'ball_tree', 'n_neighbors': 10, 'weights': 'uniform'}
Running KNN with parameters {'algorithm': 'ball_tree', 'n_neighbors': 10, 'weights': 'distance'}
Running KNN with parameters {'

Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 100}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 1000}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 100}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 1000}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 5, 'n_estimators': 100}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 5, 'n_estimators': 1000}
Running ET with parameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 10, 'n_estimators': 100}
Running ET with parameters {'criter

/home/sinclairtarget/harris/ml/philly-evictions/.venv/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


### Regression Models
Our regression models are given by the following list:

In [11]:
pipeline.regs

{'LR': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
 'SVR': LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
           intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
           random_state=None, tol=0.0001, verbose=0),
 'DTR': DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort=False, random_state=None, splitter='best'),
 'RFR': RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                       max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_esti

We plan to run a grid search using the following hyperparameters:

In [12]:
pipeline.reg_small_grid

{'LR': {},
 'SVR': {'C': [0.01, 0.1]},
 'DTR': {'max_depth': [5, 50],
  'max_features': [None],
  'min_samples_split': [2, 5, 10]},
 'RFR': {'n_estimators': [100, 1000],
  'max_depth': [5, 50],
  'max_features': ['sqrt', 'log2'],
  'min_samples_split': [2, 5, 10]}}

We run our models for each of our splits.

In [13]:
models = pipeline.regs
grid = pipeline.reg_small_grid

results_df = pd.DataFrame(columns=[
    'split',
    'classifier',
    'parameters',
] + pipeline.evaluate.RegressionEvaluator.metric_names())

for i, (train_df, test_df) in enumerate(cleaned_splits, start=1):
    train_df = train_df.drop(columns=['GEOID', 'year_evictions'])
    test_df = test_df.drop(columns=['GEOID', 'year_evictions'])
    df = pipeline.run_reg_loop(
        test_df, train_df, models, grid, 'evictions'
    )

    df = df.assign(split=i)
    results_df = results_df.append(df, ignore_index=True)

results_df.to_csv('results/reg-small-grid.csv', index=False)

/home/sinclairtarget/harris/ml/philly-evictions/.venv/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/sinclairtarget/harris/ml/philly-evictions/.venv/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/sinclairtarget/harris/ml/philly-evictions/.venv/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWa